# Colab pour l'apprentisage semi supervisé du projet IA


# Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [ ]:
!git clone https://github.com/naoutix/detectionNetwork.git  # clone


Cloning into 'detectionNetwork'...
remote: Enumerating objects: 52868, done.
remote: Counting objects: 100% (1546/1546), done.
remote: Compressing objects: 100% (574/574), done.
remote: Total 52868 (delta 246), reused 1478 (delta 188), pack-reused 51322
Receiving objects: 100% (52868/52868), 110.35 MiB | 13.46 MiB/s, done.
Resolving deltas: 100% (1783/1783), done.
Checking out files: 100% (73148/73148), done.


In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
from yolov5 import utils
display = utils.notebook_init()  # checks
!pip install roboflow
from roboflow import Roboflow

YOLOv5 🚀 v6.0-241-gf627bc5 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 42.6/78.2 GB disk)
     |████████████████████████████████| 67 kB 3.2 MB/s 
     |████████████████████████████████| 178 kB 35.8 MB/s 
     |████████████████████████████████| 145 kB 43.0 MB/s 
     |████████████████████████████████| 138 kB 45.7 MB/s 
     |████████████████████████████████| 1.1 MB 44.8 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=cf5754cc59483ab34322b4a3871decc64c032fb669456246452e2d7cac33aa9f
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.7
    Uninstalling pyparsing-3.0.7:
      Successfully uninstalled pyparsing-3.0.7
 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# /content/drive/MyDrive/Colab Notebooks/best.pt

Mounted at /content/drive


# 1. Inference

`detect.py` runs YOLOv5 inference on a variety of sources, downloading models automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases), and saving results to `runs/detect`. Example inference sources are:

```shell
python detect.py --source 0  # webcam
                          img.jpg  # image 
                          vid.mp4  # video
                          path/  # directory
                          path/*.jpg  # glob
                          'https://youtu.be/Zgi9g1ksQHc'  # YouTube
                          'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream
```

In [ ]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images
display.Image(filename='runs/detect/exp/zidane.jpg', width=600)

# 2. Validate
Validate a model's accuracy on [COCO](https://cocodataset.org/#home) val or test-dev datasets. Models are downloaded automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases). To show results by class use the `--verbose` flag. Note that `pycocotools` metrics may be ~1% better than the equivalent repo metrics, as is visible below, due to slight differences in mAP computation.

## COCO val
Download [COCO val 2017](https://github.com/ultralytics/yolov5/blob/74b34872fdf41941cddcf243951cdb090fbac17b/data/coco.yaml#L14) dataset (1GB - 5000 images), and test model accuracy.

In [ ]:
# Download COCO val
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../datasets && rm tmp.zip

  0%|          | 0.00/780M [00:00<?, ?B/s]

In [ ]:
# Run YOLOv5x on COCO val
!python val.py --weights yolov5x.pt --data coco.yaml --img 640 --iou 0.65 --half

val: data=/content/yolov5/data/coco.yaml, weights=['yolov5x.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.65, task=val, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=True
YOLOv5 🚀 v6.0-48-g84a8099 torch 1.10.0+cu102 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

100% 166M/166M [00:03<00:00, 54.1MB/s]

Fusing layers... 
Model Summary: 444 layers, 86705005 parameters, 0 gradients
val: Scanning '../datasets/coco/val2017' images and labels...4952 found, 48 missing, 0 empty, 0 corrupted: 100% 5000/5000 [00:01<00:00, 2636.64it/s]
val: New cache created: ../datasets/coco/val2017.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 157/157 [01:12<00:00,  2.17it/s]
                 all       5000      36335      0.729       0.63      0.683      0.496
Speed: 0.1ms pre-process, 4.9ms inference, 1.9ms NMS per image a

## COCO test
Download [COCO test2017](https://github.com/ultralytics/yolov5/blob/74b34872fdf41941cddcf243951cdb090fbac17b/data/coco.yaml#L15) dataset (7GB - 40,000 images), to test model accuracy on test-dev set (**20,000 images, no labels**). Results are saved to a `*.json` file which should be **zipped** and submitted to the evaluation server at https://competitions.codalab.org/competitions/20794.

In [ ]:
# Download COCO test-dev2017
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017labels.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../datasets && rm tmp.zip
!f="test2017.zip" && curl http://images.cocodataset.org/zips/$f -o $f && unzip -q $f -d ../datasets/coco/images

In [ ]:
# Run YOLOv5x on COCO test
!python val.py --weights yolov5x.pt --data coco.yaml --img 640 --iou 0.65 --half --task test

# 3. Train


Train a YOLOv5s model on the [COCO128](https://www.kaggle.com/ultralytics/coco128) dataset with `--data coco128.yaml`, starting from pretrained `--weights yolov5s.pt`, or from randomly initialized `--weights '' --cfg yolov5s.yaml`.

- **Pretrained [Models](https://github.com/ultralytics/yolov5/tree/master/models)** are downloaded
automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases)
- **[Datasets](https://github.com/ultralytics/yolov5/tree/master/data)** available for autodownload include: [COCO](https://github.com/ultralytics/yolov5/blob/master/data/coco.yaml), [COCO128](https://github.com/ultralytics/yolov5/blob/master/data/coco128.yaml), [VOC](https://github.com/ultralytics/yolov5/blob/master/data/VOC.yaml), [Argoverse](https://github.com/ultralytics/yolov5/blob/master/data/Argoverse.yaml), [VisDrone](https://github.com/ultralytics/yolov5/blob/master/data/VisDrone.yaml), [GlobalWheat](https://github.com/ultralytics/yolov5/blob/master/data/GlobalWheat2020.yaml), [xView](https://github.com/ultralytics/yolov5/blob/master/data/xView.yaml), [Objects365](https://github.com/ultralytics/yolov5/blob/master/data/Objects365.yaml), [SKU-110K](https://github.com/ultralytics/yolov5/blob/master/data/SKU-110K.yaml).
- **Training Results** are saved to `runs/train/` with incrementing run directories, i.e. `runs/train/exp2`, `runs/train/exp3` etc.
<br><br>


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="hkPgaOuEiMRONK7xCyar")
project = rf.workspace().project("projet-ia")
dataset = project.version(4).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Projet-IA-4 in yolov5pytorch:: 100%|██████████| 4871/4871 [00:01<00:00, 2461.98it/s]


In [ ]:
# Train YOLOv5s on ANIMALS DATASET for 50 epochs
!python train.py --img 64 --batch 20 --epochs 40 --data /content/yolov5/IA_detection-1/data.yaml --weights yolov5m.pt

# 4. Visualize

# 5.Teacher student
## génération des pseudo-labels

In [ ]:
rf = Roboflow(api_key="hkPgaOuEiMRONK7xCyar")
project = rf.workspace().project("projet-ia-non-labellise")
dataset = project.version(4).download("yolov5")


loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Projet-IA-non-labellisé-4 in yolov5pytorch:: 100%|██████████| 18826/18826 [00:07<00:00, 2634.39it/s]


In [ ]:
!ls /content/yolov5/Projet-IA-non-labellisé-4/valid/images/ | wc -l
# !rm -rf /content/yolov5/Projet-IA-non-labellisé-4/
!ls /content/yolov5/Projet-IA-non-labellisé-4/valid/labels | wc -l

9410
9410


In [ ]:
!python val.py --conf-thres 0.9 --save-hybrid --weights /content/detectionNetwork/meilleur_entrainement/exp12/weights/best.pt --img 64 --data /content/yolov5/Projet-IA-non-labellisé-4/data.yaml

val: data=/content/yolov5/Projet-IA-non-labellisé-4/data.yaml, weights=['/content/detectionNetwork/meilleur_entrainement/exp12/weights/best.pt'], batch_size=32, imgsz=64, conf_thres=0.9, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=True, save_hybrid=True, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.0-241-gf627bc5 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 213 layers, 7031701 parameters, 0 gradients, 15.9 GFLOPs
val: Scanning '/content/yolov5/Projet-IA-non-labellisé-4/valid/labels' images and labels...9410 found, 0 missing, 9410 empty, 0 corrupt: 100% 9410/9410 [00:04<00:00, 2086.49it/s]
val: New cache created: /content/yolov5/Projet-IA-non-labellisé-4/valid/labels.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 295/295 [00:20<00:00, 14.55it/s]
                 all       9410

## Fusion des label et pseudo-label

In [ ]:
!rm -rf /content/yolov5/IA_detection-TS
!cp -r /content/yolov5/Projet-IA-4/ /content/yolov5/IA_detection-TS
!ls /content/yolov5/IA_detection-TS/train/images/ | wc -l


2213


In [ ]:
!cp -r /content/yolov5/Projet-IA-non-labellisé-4/valid/* /content/yolov5/IA_detection-TS/train/
!ls /content/yolov5/IA_detection-TS/train/labels | wc -l

11623


## Apprentissage du student 

In [ ]:
# Train YOLOv5s on ANIMALS DATASET for 50 epochs
!python train.py --img 64 --batch 20 --epochs 60 --data /content/yolov5/IA_detection-TS/data.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=/content/yolov5/IA_detection-TS/data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=60, batch_size=20, imgsz=64, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 5 (delta 1), reused 5 (delta 1), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/ultralytics/yolov5
   f627bc5..f40854b  master     -> origin/master
   2a20f8c..b4b5c3a  classifier -> origin/classifier
github: ⚠️ YOLOv5 is out of date by 

## Tentative de tranfert learning sur le student

(Peu concluant)

In [ ]:
!python train.py --img 64 --batch 8 --epochs 60 --data /content/yolov5/IA_detection-TS/data.yaml --freeze 10

train: weights=yolov5s.pt, cfg=, data=/content/yolov5/IA_detection-TS/data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=60, batch_size=8, imgsz=64, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[10], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 1 commit. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.0-241-gf627bc5 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.

In [47]:
!python train.py --img 64 --batch 8 --epochs 60 --data /content/yolov5/IA_detection-TS/data.yaml --weights /content/yolov5/runs/train/exp3/weights/best.pt --hyp /content/detectionNetwork/data/hyps/hyp.finetune.yaml

train: weights=/content/yolov5/runs/train/exp3/weights/best.pt, cfg=, data=/content/yolov5/IA_detection-TS/data.yaml, hyp=/content/detectionNetwork/data/hyps/hyp.finetune.yaml, epochs=60, batch_size=8, imgsz=64, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 1 commit. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.0-241-gf627bc5 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.0032, lrf=0.12, momentum=0.843, weight_decay=0.00036, warmup_epochs=2.0, warmup_momentum=0.5, warmup_bias_lr=0.05, box=0.

In [ ]:
from matplotlib import pyplot as plt 
import cv2
import pandas as pd 
def plot_results(confusion_matrix, metriques, file_csv, val_pred_0, val_pred_1):
  # Matrice de confusion
  cm = cv2.imread(confusion_matrix)
  # Metriques
  metriques = cv2.imread(metriques)
  # CSV file
  headers = ['epoch', 'train/box_loss', 'train/cls_loss', 'precision', 'recall', 'mAP_0.5', 'mAP_0.95', 'val/box_loss', 'val/cls_loss', 'x/lr0', 'x/lr1', 'x/lr2']
  df = pd.read_csv(file_csv)
  fig = plt.figure(figsize=(15, 15)) 
  # Prediction sur les images de validation
  val_pred_0 = cv2.imread(val_pred_0)
  val_pred_1 = cv2.imread(val_pred_1)
  
  rows = 2
  columns = 2
    
  fig.add_subplot(rows, columns, 1) 
    
  plt.imshow(cm) 
  plt.axis('off')
  plt.title("Labels") 
    
  fig.add_subplot(rows, columns, 2) 
    
  plt.imshow(metriques) 
  plt.axis('off') 
  plt.title("Metriques")

  fig.add_subplot(rows, columns, 3) 
    
  plt.imshow(val_pred_0) 
  plt.axis('off') 
  plt.title("Validation_1")

  fig.add_subplot(rows, columns, 4) 
    
  plt.imshow(val_pred_1) 
  plt.axis('off') 
  plt.title("Validation_2")
  return df

In [ ]:
path= '/content/yolov5/runs/train/exp/'
path= '/content/detectionNetwork/meilleur_entrainement/exp12/'
path= '/content/yolov5/runs/train/exp4/'
val_pred_0 = path + 'val_batch0_pred.jpg'
val_pred_1 = path + 'val_batch1_pred.jpg'
file_csv = path + "results.csv"
cm = path + "confusion_matrix.png"
metriques = path + "results.png"


df = plot_results(confusion_matrix=cm,metriques=metriques,file_csv=file_csv, val_pred_0=val_pred_0, val_pred_1=val_pred_1)

In [ ]:
df.iloc[:,4:8].describe()


,metrics/precision,metrics/recall,metrics/mAP_0.5,metrics/mAP_0.5:0.95
count,60.000000,60.000000,60.000000,60.000000
mean,0.377066,0.462336,0.343658,0.157057
std,0.164508,0.104185,0.105999,0.054175
min,0.010099,0.026396,0.007057,0.002209
25%,0.264508,0.410050,0.285845,0.120902
50%,0.347635,0.467655,0.378895,0.170520
75%,0.498493,0.529465,0.429875,0.193480
max,0.711210,0.630200,0.465910,0.239330
